In [21]:
import os
import pickle
import asyncio
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
import gradio as gr

# Check environment variable
deployment_name = os.getenv("OPENAI_API_DEPLOYMENT_NAME")
if not deployment_name:
    raise ValueError("OPENAI_API_DEPLOYMENT_NAME environment variable is not set.")
print(f"Using deployment name: {deployment_name}")

# Initialize the model
model = AzureChatOpenAI(deployment_name=deployment_name)

# Function to read file contents
def get_file_content(filepath):
    try:
        with open(filepath, 'r') as file:
            return file.read()
    except FileNotFoundError:
        print(f"File {filepath} not found.")
        return ""

# Manage session history
store = {}
def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Function to get system prompt
def get_system_prompt():
    return get_file_content('./SystemPrompt-VSC-BusinesCentral-TestRunner.txt')

file_paths = {
    'al_endpoint': '../BusinessCentral-AL-Test-Runner-App/src/codeunit/WebApi.Codeunit.al',
    'package_json': '../VSC-BusinessCentral-Test-Runner/package.json',
    'apiclient_ts': '../VSC-BusinessCentral-Test-Runner/src/apiClient.ts',
    'project_ts': '../VSC-BusinessCentral-Test-Runner/src/project.ts',
    'testrunner_ts': '../VSC-BusinessCentral-Test-Runner/src/testRunner.ts',
    'testviewproviders_ts': '../VSC-BusinessCentral-Test-Runner/src/testViewProviders.ts',
    'extension_ts': '../VSC-BusinessCentral-Test-Runner/src/extension.ts',
}

file_contents = {key: get_file_content(path) for key, path in file_paths.items()}
systemprompt = get_system_prompt()

# Escape the curly braces
def escape_curly_braces(s):
    return s.replace('{', '{{').replace('}', '}}')

system_message = SystemMessage(content=systemprompt)
file_content_messages = [
    SystemMessage(content=f"#### This is the content of {key}: {escape_curly_braces(file_contents[key])}\n---")
    for key in file_contents
]
user_message_template = HumanMessage(content="{input}")
messages = [system_message, *file_content_messages, user_message_template]

prompt_template = ChatPromptTemplate.from_messages(messages)
print("Prompt template created.")

konnos_chat_history = get_session_history("default")
current_file_path = None
print("Chat history initialized.")

def save_chat_history_as(file_path):
    global current_file_path
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(konnos_chat_history.messages, f)
        print(f"Chat history saved as: {file_path}")
        current_file_path = file_path
        return file_path
    except Exception as e:
        print(f"Failed to save chat history: {e}")
        return None

def save_chat_history():
    global current_file_path
    if current_file_path is None:
        return None
    try:
        with open(current_file_path, 'wb') as f:
            pickle.dump(konnos_chat_history.messages, f)
        print(f"Chat history saved to: {current_file_path}")
        return current_file_path
    except Exception as e:
        print(f"Failed to save chat history: {e}")
        return None

def load_chat_history(file_path):
    global konnos_chat_history, current_file_path
    try:
        with open(file_path, 'rb') as f:
            konnos_chat_history.messages = pickle.load(f)
        current_file_path = file_path
        chat_entries = [(msg.content if isinstance(msg, HumanMessage) else "", msg.content if isinstance(msg, AIMessage) else "") for msg in konnos_chat_history.messages]
        print(f"Chat history loaded from: {file_path}")
        return chat_entries
    except Exception as e:
        print(f"Failed to load chat history: {e}")
        return []

async def converse(user_input):
    print(f"User input received: {user_input}")
    chat_history_list = konnos_chat_history.messages

    formatted_prompt = prompt_template.format(
        chat_history=chat_history_list,
        input=user_input,
        **file_contents
    )

    konnos_chat_history.add_message(HumanMessage(content=user_input))
    print("Human message added to history.")
    
    response = await asyncio.to_thread(model.invoke, formatted_prompt)
    konnos_chat_history.add_message(AIMessage(content=response.content))
    print("AI message added to history.")
    return response.content

async def interactive_conversation_stream(user_input):
    chat_history_list = konnos_chat_history.messages

    formatted_prompt = prompt_template.format(
        chat_history=chat_history_list,
        input=user_input,
        **file_contents
    )

    konnos_chat_history.add_message(HumanMessage(content=user_input))
    print("Human message added to history.")

    response = await asyncio.to_thread(model.invoke, formatted_prompt)

    konnos_chat_history.add_message(AIMessage(content=response.content))
    print("AI message added to history.")
    
    for char in response.content:
        yield char
        
def delete_message(index):
    if 0 <= index < len(konnos_chat_history.messages):
        del konnos_chat_history.messages[index]
        print(f"Deleted message at index {index}")

def delete_all_messages():
    konnos_chat_history.messages.clear()
    print("All messages deleted.")

# Gradio UI
with gr.Blocks(css=".small-button { max-width: 100px; display: inline-block !important; margin-right: 10px; }") as demo:
    chatbot = gr.Chatbot()
    message_input = gr.Textbox(placeholder="Type your message here...", label="You")

    with gr.Row():
        with gr.Column(scale=1):
            save_button = gr.Button("Save", variant="primary")
        with gr.Column(scale=1):
            save_as_button = gr.Button("Save As", variant="secondary")
        with gr.Column(scale=1):
            open_file = gr.File(label="Open Chat (.pkl)", type="filepath", file_count="single")
        with gr.Column(scale=1):
            delete_all_button = gr.Button("Delete All", variant="warning")

    async def respond(message, chat_history):
        print(f"Responding to user message: {message}")
        bot_response = "".join([chunk async for chunk in interactive_conversation_stream(message)]).strip()
        print(f"Bot response: {bot_response}")
        chat_history.append((message, bot_response))
        return chat_history, ""

    def trigger_save_as_dialog():
        return gr.File(label="Save As", type="filepath", file_count="single")

    def handle_save_as_click(file):
        if file:
            save_chat_history_as(file.name)
        return gr.update()

    def handle_save_click():
        global current_file_path
        if current_file_path is None:
            # Trigger 'Save As' dialog if no file path is set
            return trigger_save_as_dialog()
        else:
            save_chat_history()
        return gr.update()

    def handle_open_file(file):
        if file:
            chat_history = load_chat_history(file.name)
            return chat_history
        return []

    def handle_delete_all_click():
        delete_all_messages()
        return gr.update()

    save_button.click(fn=handle_save_click, inputs=[], outputs=[])
    save_as_button.click(fn=trigger_save_as_dialog, inputs=[], outputs=[save_as_button])
    #save_as_button.upload(fn=handle_save_as_click, inputs=save_as_button, outputs=[])
    open_file.upload(fn=handle_open_file, inputs=open_file, outputs=[chatbot])
    delete_all_button.click(fn=handle_delete_all_click, inputs=[], outputs=[])

    message_input.submit(respond, [message_input, chatbot], [chatbot, message_input])

demo.launch()

Using deployment name: gpt-4o
Prompt template created.
Chat history initialized.


AttributeError: 'Button' object has no attribute 'upload'

In [23]:
import gradio as gr
from azure.ai.openai import OpenAIClient
from azure.core.credentials import AzureKeyCredential
import json
from time import sleep

# Azure OpenAI und LangChain initialisieren
azure_api_key = 'YOUR_AZURE_API_KEY'
azure_api_endpoint = 'YOUR_AZURE_API_ENDPOINT'
client = OpenAIClient(endpoint=azure_api_endpoint, credential=AzureKeyCredential(azure_api_key))

chat_history = []

def query_openai(prompt):
    response = client.completions.create(
        prompt=prompt,
        max_tokens=150,
        temperature=0.9,
    )
    return response.choices[0].text.strip()

def stream_response(prompt):
    response_text = query_openai(prompt)
    for i in range(1, len(response_text)+1):
        yield response_text[:i]
        sleep(0.05)

def chat(input_text, chat_history):
    chat_history.append(("User", input_text))
    response_stream = stream_response(input_text)
    response_text = ""

    for chunk in response_stream:
        response_text = chunk
        yield chat_history + [("AI", response_text)], response_text

def save_chat():
    with open('chat_history.json', 'w') as f:
        json.dump(chat_history, f)
    return "Chat gespeichert!"

def save_chat_as(file_path):
    with open(file_path, 'w') as f:
        json.dump(chat_history, f)
    return f"Chat gespeichert als: {file_path}"

def open_chat(file_path):
    global chat_history
    with open(file_path, 'r') as f:
        chat_history = json.load(f)
    return chat_history

with gr.Blocks() as demo:
    chatbox = gr.Chatbot(label="Chatbot mit Azure OpenAI und LangChain")
    user_input = gr.Textbox(label="Deine Nachricht")
    submit_button = gr.Button("Senden")
    save_button = gr.Button("Speichern")
    save_as_button = gr.Button("Speichern als")
    open_button = gr.Button("Öffnen")
    file_picker = gr.File(label="Datei auswählen", type="file")

    output_text = gr.Textbox(label="AI Antwort")
    progress = gr.HTML("")

    submit_button.click(chat, [user_input, chatbox], [chatbox, output_text])
    save_button.click(save_chat, [], progress)
    save_as_button.click(save_chat_as, [file_picker], progress)
    open_button.click(open_chat, [file_picker], chatbox)

demo.launch()


TypeError: EventListener._setup.<locals>.event_trigger() got an unexpected keyword argument 'live'